In [1]:
# importing packages
#!pip install selenium
import pandas as pd
import re

from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
start_time = time()

from warnings import warn

In [2]:
# replace variables here.
url = "https://www.linkedin.com/jobs/search/?geoId=90010383&keywords=data%20analyst&location=Amsterdam%20Area"
#url = "https://www.linkedin.com/jobs/search/?f_TPR=r604800&geoId=101174742&keywords=data%20analyst&location=Canada&sortBy=DD"
no_of_jobs = 2

In [4]:
# this will open up new window with the url provided above 
#download  https://sites.google.com/a/chromium.org/chromedriver/downloads
driver = webdriver.Chrome('C:/Users/poche/OneDrive/Desktop/chromedriver')
driver.get(url)
sleep(3)
action = ActionChains(driver)

In [5]:
# to show more jobs. Depends on number of jobs selected
i = 2
while i <= (no_of_jobs/25): 
    driver.find_element_by_xpath('/html/body/main/div/section/button').click()
    i = i + 1
    sleep(5)

In [6]:
# parsing the visible webpage
pageSource = driver.page_source
lxml_soup = BeautifulSoup(pageSource, 'lxml')

# searching for all job containers
job_container = lxml_soup.find('ul', class_ = 'jobs-search__results-list')

print('You are scraping information about {} jobs.'.format(len(job_container)))

You are scraping information about 25 jobs.


In [7]:
# setting up list for job information
job_id = []
post_title = []
company_name = []
post_date = []
job_location = []
job_desc = []
level = []
emp_type = []
functions = []
industries = []

# for loop for job title, company, id, location and date posted
for job in job_container:
    
    # job title
    job_titles = job.find("span", class_="screen-reader-text").text
    post_title.append(job_titles)
    
    # linkedin job id
    job_ids = job.find('a', href=True)['href']
    job_ids = re.findall(r'(?!-)([0-9]*)(?=\?)',job_ids)[0]
    job_id.append(job_ids)
    
    # company name
    company_names = job.select_one('img')['alt']
    company_name.append(company_names)
    
    # job location
    job_locations = job.find("span", class_="job-result-card__location").text
    job_location.append(job_locations)
    
    # posting date
    post_dates = job.select_one('time')['datetime']
    post_date.append(post_dates)

# for loop for job description and criterias
for x in range(1,len(job_id)+1):
    
    # clicking on different job containers to view information about the job
    job_xpath = '/html/body/main/div/section/ul/li[{}]/img'.format(x)
    driver.find_element_by_xpath(job_xpath).click()
    sleep(3)
    
    # job description
    jobdesc_xpath = '/html/body/main/section/div[2]/section[2]/div'
    job_descs = driver.find_element_by_xpath(jobdesc_xpath).text
    job_desc.append(job_descs)
    
    # job criteria container below the description
    job_criteria_container = lxml_soup.find('ul', class_ = 'job-criteria__list')
    all_job_criterias = job_criteria_container.find_all("span", class_='job-criteria__text job-criteria__text--criteria')
    
    # Seniority level
    seniority_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[1]'
    seniority = driver.find_element_by_xpath(seniority_xpath).text.splitlines(0)[1]
    level.append(seniority)
    
    # Employment type
    type_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[2]'
    employment_type = driver.find_element_by_xpath(type_xpath).text.splitlines(0)[1]
    emp_type.append(employment_type)
    
    # Job function
    function_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[3]'
    job_function = driver.find_element_by_xpath(function_xpath).text.splitlines(0)[1]
    functions.append(job_function)
    
    # Industries
    industry_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[4]'
    industry_type = driver.find_element_by_xpath(industry_xpath).text.splitlines(0)[1]
    industries.append(industry_type)
    
    x = x+1

In [8]:
# to check if we have all information
print(len(job_id))
print(len(post_date))
print(len(company_name))
print(len(post_title))
print(len(job_location))
print(len(job_desc))
print(len(level))
print(len(emp_type))
print(len(functions))
print(len(industries))

25
25
25
25
25
25
25
25
25
25


In [9]:
# creating a dataframe
job_data = pd.DataFrame({'Job ID': job_id,
'Date': post_date,
'Company Name': company_name,
'Post': post_title,
'Location': job_location,
'Description': job_desc,
'Level': level,
'Type': emp_type,
'Function': functions,
'Industry': industries
})

# cleaning description column
job_data['Description'] = job_data['Description'].str.replace('\n',' ')

print(job_data.info())
job_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Job ID        25 non-null     object
 1   Date          25 non-null     object
 2   Company Name  25 non-null     object
 3   Post          25 non-null     object
 4   Location      25 non-null     object
 5   Description   25 non-null     object
 6   Level         25 non-null     object
 7   Type          25 non-null     object
 8   Function      25 non-null     object
 9   Industry      25 non-null     object
dtypes: object(10)
memory usage: 2.1+ KB
None


,Job ID,Date,Company Name,Post,Location,Description,Level,Type,Function,Industry
0,2463163321,2021-03-16,PepsiCo Graphic,Junior Data Analyst,"Utrecht, Utrecht, Netherlands",Auto req ID: 229888BR Job Description The Co...,Associate,Full-time,Sales,Consumer GoodsFood & Beverages
1,2486699987,2021-04-02,Genpact Graphic,Business Data Analyst,Amsterdam Area,"With a startup spirit and 90,000+ curious and ...",Mid-Senior level,Full-time,Information Technology,Information Technology and Services
2,2485374771,2021-03-31,eBay Graphic,Data Analyst,"Amsterdam, North Holland, Netherlands",eBay Classifieds Group (eCG) is an innovative ...,Entry level,Part-time,Information Technology,Computer SoftwareInternetFinancial Services
3,2478321840,2021-04-06,Zoom Graphic,Data Analyst / Business Insights Analyst EMEA,"Amsterdam, North Holland, Netherlands",Zoomies help the world connect — and deliver h...,Not Applicable,Full-time,Information Technology,Information Technology and ServicesComputer So...
4,2477283218,2021-04-06,Tony&amp;amp;#39;s Chocolonely Graphic,Jr.Business Analyst,"Amsterdam, North Holland, Netherlands",Direct message the job poster from Tony's Choc...,Entry level,Full-time,AnalystConsultingStrategy/Planning,Food & Beverages


In [10]:
#from genism.corpora.dictionary import Dictionary
#nltk.download('stopwords')
from nltk import sent_tokenize , word_tokenize
from nltk.corpus import stopwords
#print(stopwords.words('english'))
english_stops = stopwords.words('english')
#remove stop words 

train = job_data.iloc[0,5]
words = word_tokenize(train)
no_stops = [t for t in words if t not in english_stops]
print(no_stops)



['Auto', 'req', 'ID', ':', '229888BR', 'Job', 'Description', 'The', 'Commercial', 'Data', 'Analyst', 'Benelux', 'unique', 'role', 'combining', 'business', 'acumen', 'consultancy', 'technical', 'expertise', '.', 'The', 'role', 'sits', 'Analytics', 'Impact', '(', 'AI', ')', 'team', 'whose', 'purpose', 'deliver', 'business', 'impact', 'data', 'industry-leading', 'analytics', ',', 'close', 'collaboration', 'PepsiCo', '’', 'commercial', 'departments', ':', 'DX', '(', 'Demand', 'Accelerator', ')', ',', 'Sales', ',', 'Marketing', '.', 'The', 'Commercial', 'Data', 'Analyst', 'support', 'PepsiCo', 'Benelux', 'Commercial', 'Data', '&', 'Analytics', 'strategy', 'definition', '.', 'Together', '3', 'members', 'AI', 'team', ',', 'he/she', 'co-own', 'data', 'management', 'process', ',', 'ensuring', 'prioritize', ',', 'collect', ',', 'store', 'treat', 'data', 'right', 'way', '.', 'He/she', 'play', 'instrumental', 'role', 'data', 'democratization', ',', 'making', 'sure', 'data', 'insights', 'available'